# Lab 3_2

## 2. Credit card fraud data. 
Download the data set at

https://www.kaggle.com/mlg-ulb/creditcardfraud.

This data set has details of 284,807 credit card transactions, some of which are fraudulent. Each transaction is represented by 28 features (scrambled using PCA as a primitive kind of anonymization), and has a corresponding label (1 is fraudulent and 0 is legitimate).

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from pandas import read_csv
from matplotlib.pyplot import figure
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
import graphviz
from sklearn import tree
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [2]:
data = read_csv('creditcard.csv')
data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


## (a) How many of the transactions are fraudulent? Why might this be problematic when learning a classifier?

In [3]:
df_data = pd.DataFrame(data = data)
num = df_data[df_data['Class'] == 1]['Class'].count()
print('There are', num, 'transactions are fraudulent.')

There are 492 transactions are fraudulent.


The dataset for fraud data is too small compared to the legitimate data. If we are learning a classifier, it might overfit.

## (b) Downsample the legitimate transactions to make the data set more balanced.

In [4]:
from sklearn.model_selection import train_test_split
fraud_df = df_data[df_data['Class'] == 1]
legit_df = df_data[df_data['Class'] == 0]
legit_data_df = legit_df.sample(n=492, axis=0)
df = pd.DataFrame(fraud_df)
df = df.append(legit_data_df)
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
541,406.0,-2.312227,1.951992,-1.609851,3.997906,-0.522188,-1.426545,-2.537387,1.391657,-2.770089,...,0.517232,-0.035049,-0.465211,0.320198,0.044519,0.177840,0.261145,-0.143276,0.00,1
623,472.0,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,...,0.661696,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,529.00,1
4920,4462.0,-2.303350,1.759247,-0.359745,2.330243,-0.821628,-0.075788,0.562320,-0.399147,-0.238253,...,-0.294166,-0.932391,0.172726,-0.087330,-0.156114,-0.542628,0.039566,-0.153029,239.93,1
6108,6986.0,-4.397974,1.358367,-2.592844,2.679787,-1.128131,-1.706536,-3.496197,-0.248778,-0.247768,...,0.573574,0.176968,-0.436207,-0.053502,0.252405,-0.657488,-0.827136,0.849573,59.00,1
6329,7519.0,1.234235,3.019740,-4.304597,4.732795,3.624201,-1.357746,1.713445,-0.496358,-1.282858,...,-0.379068,-0.704181,-0.656805,-1.632653,1.488901,0.566797,-0.010016,0.146793,1.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85077,60592.0,-3.559172,1.854252,0.959631,2.302952,-0.022688,1.387984,-0.495071,0.688798,1.169169,...,-0.408164,0.306582,0.074103,-1.343625,0.782817,0.550981,0.767263,0.573870,0.76,0
261003,159819.0,-0.168992,0.925731,-0.127012,-0.795499,0.685106,-0.677470,0.934875,-0.093816,0.285683,...,-0.337176,-0.749606,0.017753,-0.741489,-0.377080,0.193617,0.350631,0.154924,8.94,0
240739,150726.0,-13.099603,-25.434451,-11.921912,4.976147,-6.139222,3.203290,10.923104,-2.020641,-0.498651,...,5.092726,-2.970344,-7.083262,-0.439702,-2.386293,0.159206,-1.588007,1.245735,7862.39,0
97593,66289.0,-1.115645,0.329991,1.170213,-0.301904,0.442068,-1.702710,1.106799,-0.172549,-0.529010,...,-0.220795,-0.964776,0.349180,0.655277,-0.220789,-0.047188,0.219880,0.197307,89.90,0


## (c) Fit three kinds of classifier to the data:

• decision tree

• boosted decision stumps 

• random forest

In each case, use cross-validation to estimate the confusion matrix.

In [5]:
inputs = df.values[:, 1:29]
labels = df.values[:, -1]

In [6]:
clf = DecisionTreeClassifier(random_state=0, criterion='gini')
clf.fit(inputs, labels)
pred = cross_val_predict(clf, inputs, labels, cv=10)
conf_mat = confusion_matrix(labels, pred)
conf_mat

array([[448,  44],
       [ 39, 453]])

In [7]:
clf_a = AdaBoostClassifier(random_state=0)
clf_a.fit(inputs, labels)
pred_a = cross_val_predict(clf_a, inputs, labels, cv=10)
conf_mat_a = confusion_matrix(labels, pred_a)
conf_mat_a

array([[466,  26],
       [ 40, 452]])

In [12]:
clf_r = RandomForestClassifier(random_state=0)
clf_r.fit(inputs, labels)
pred_r = cross_val_predict(clf_r, inputs, labels, cv=10)
conf_mat_r = confusion_matrix(labels, pred_r)
conf_mat_r

array([[476,  16],
       [ 44, 448]])